In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from io import StringIO
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import clear_output
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics
import keyboard
import time

def millis():
    return time.time_ns() / 1000000
oldtime = millis()
curtime = 0
rectime = 0
step = 1/60 * 1000 #60 hz

## Lectura de datos

In [2]:
# Leer carpeta
CARPETA = "datos_cluster2d"
datos = False
columnas = [
    # Comandos
    "throttle_input",
    "brake_input",
    "clutch_input",
    "steering_input",
    "gear",
    # Estado
    "rpm",
    "airspeed"
]
files = Path(CARPETA).glob('*')
for i, ruta in enumerate(files):
    df = pd.read_csv(ruta, usecols=columnas + ["is_shifting"])
    if (i == 0):
        datos = df
    else:
        datos = pd.concat([datos, df], ignore_index=True)
# MANTENER CAMBIO
gear = 0
for i in range(datos.shape[0]):
    if (datos["is_shifting"].iloc[i] == True):
        if (datos["gear"].iloc[i] == 0):
            datos.at[i, "gear"] = gear
    else:
        gear = datos["gear"].iloc[i]
datos.drop(columns="is_shifting", inplace=True)
# OBTENER ACELERACIÓN
accel = [0]
for i in range(datos.shape[0]):
    if i == 0:
        continue
    accel.append(datos["airspeed"].iloc[i] - datos["airspeed"].iloc[i-1])
datos["accel"] = accel
datos.drop(datos[datos["accel"]<-0.2].index, inplace=True)
display(datos)

,airspeed,steering_input,gear,rpm,brake_input,throttle_input,clutch_input,accel
0,0.000151,0.000000,0.0,670.807550,0.0,0.0,0.0,0.000000
1,0.000133,0.000000,0.0,667.582291,0.0,0.0,0.0,-0.000018
2,0.000143,0.000000,0.0,664.387431,0.0,0.0,0.0,0.000010
3,0.000144,0.000000,0.0,661.140019,0.0,0.0,0.0,0.000002
4,0.000142,0.000000,0.0,664.203268,0.0,0.0,0.0,-0.000002
...,...,...,...,...,...,...,...,...
37039,19.755055,0.392376,3.0,2750.780320,0.0,0.0,0.0,-0.122945
37040,19.628862,0.291094,3.0,2688.020298,0.0,0.0,0.0,-0.126193
37041,19.498466,0.199892,3.0,2617.569343,0.0,0.0,0.0,-0.130396
37042,19.364819,0.115181,3.0,2541.855740,0.0,0.0,0.0,-0.133648


## Función para interpretar (Estado a intención)

In [3]:
def stateToCmd(x):
    cmd = 0
    throttle = x["throttle_input"].iloc[0]
    brake = x["brake_input"].iloc[0]
    clutch = x["clutch_input"].iloc[0]
    speed = x["airspeed"].iloc[0]
    accel = x["accel"].iloc[0]
    rpm = x["rpm"].iloc[0]
    if accel > 0.01:
        cmd = 1
    elif (accel >= -0.01 and accel <= 0.01):
        cmd = 0
        if (clutch > 0 and throttle > 0):
            cmd = 1
        elif clutch > 0:
            cmd = -1
    elif accel < -0.01:
        cmd = -1
        if throttle > 0:
            cmd = 1
        elif brake > 0:
            if speed > 1:
                cmd = -1
    return cmd
cmds = []
for i in range(datos.shape[0]):
    cmds.append(stateToCmd(datos[i:i+1]))
datos["cmd"] = cmds
# AGREGAR INTREPRETACIÓN A DATOS
datos = datos[columnas+["accel","cmd"]]
display(datos)

,throttle_input,brake_input,clutch_input,steering_input,gear,rpm,airspeed,accel,cmd
0,0.0,0.0,0.0,0.000000,0.0,670.807550,0.000151,0.000000,0
1,0.0,0.0,0.0,0.000000,0.0,667.582291,0.000133,-0.000018,0
2,0.0,0.0,0.0,0.000000,0.0,664.387431,0.000143,0.000010,0
3,0.0,0.0,0.0,0.000000,0.0,661.140019,0.000144,0.000002,0
4,0.0,0.0,0.0,0.000000,0.0,664.203268,0.000142,-0.000002,0
...,...,...,...,...,...,...,...,...,...
37039,0.0,0.0,0.0,0.392376,3.0,2750.780320,19.755055,-0.122945,-1
37040,0.0,0.0,0.0,0.291094,3.0,2688.020298,19.628862,-0.126193,-1
37041,0.0,0.0,0.0,0.199892,3.0,2617.569343,19.498466,-0.130396,-1
37042,0.0,0.0,0.0,0.115181,3.0,2541.855740,19.364819,-0.133648,-1


## Normalización

In [4]:
scaler = MaxAbsScaler().fit(datos)
datos_scaled = pd.DataFrame(scaler.transform(datos), index=datos.index, columns=datos.columns)

## Entrada/Salida y perceptrón multicapa

In [5]:
inp = ["steering_input", "rpm", "airspeed", "accel", "cmd"]
out = ["throttle_input", "brake_input", "clutch_input", "gear", ]

X = datos_scaled[inp]
Y = datos_scaled[out]
mlp = MLPRegressor(hidden_layer_sizes=(100,100,100,100,100,100,100,100,100), random_state=0, verbose=True)
mlp.fit(X, Y)

Iteration 1, loss = 0.02332559
Iteration 2, loss = 0.01355395
Iteration 3, loss = 0.01257816
Iteration 4, loss = 0.01205146
Iteration 5, loss = 0.01178906
Iteration 6, loss = 0.01157667
Iteration 7, loss = 0.01140791
Iteration 8, loss = 0.01132846
Iteration 9, loss = 0.01102375
Iteration 10, loss = 0.01094371
Iteration 11, loss = 0.01073683
Iteration 12, loss = 0.01066387
Iteration 13, loss = 0.01064029
Iteration 14, loss = 0.01047614
Iteration 15, loss = 0.01042142
Iteration 16, loss = 0.01044532
Iteration 17, loss = 0.01029242
Iteration 18, loss = 0.01024525
Iteration 19, loss = 0.01035623
Iteration 20, loss = 0.01008161
Iteration 21, loss = 0.01001611
Iteration 22, loss = 0.00998000
Iteration 23, loss = 0.01003948
Iteration 24, loss = 0.00989423
Iteration 25, loss = 0.00983194
Iteration 26, loss = 0.00987887
Iteration 27, loss = 0.00982770
Iteration 28, loss = 0.00965022
Iteration 29, loss = 0.00976200
Iteration 30, loss = 0.00969816
Iteration 31, loss = 0.00954631
Iteration 32, los

MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100, 100, 100),
             random_state=0, verbose=True)

## Verificación

In [6]:
bng_path = "C:\Program Files (x86)\Steam\steamapps\common\BeamNG.drive"
bng = BeamNGpy('localhost', 64256, home=bng_path)
bng.open()

Error connecting to BeamNG.tech. 25 tries left.
[WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión
Traceback (most recent call last):
  File "C:\Users\asd\miniconda3\lib\site-packages\beamngpy\connection.py", line 149, in connect_to_beamng
    self.skt.connect((self.host, self.port))
ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión


In [ ]:
oldspeed = 0
oldgear = 0
def lectura_sensores():
    global oldspeed
    global oldgear
    # LECTURA
    vehicle.poll_sensors()
    electrics.data.pop("wheelThermals")            
    x = pd.DataFrame(electrics.data.values(), index=electrics.data.keys()).transpose()
    x = x[columnas]
    # MANTENER CAMBIO
    if electrics.data["is_shifting"]:
        if electrics.data["gear"] == 0:
            x.at[0, "gear"] = oldgear
    else:
        oldgear = electrics.data["gear"]
    # OBTENER ACELERACIÓN
    curspeed = x["airspeed"].iloc[0]
    x["accel"] = curspeed - oldspeed
    oldspeed = curspeed
    return x

oldtime = 0
while True:
    cmd = 0
    if (keyboard.is_pressed('D')):
        break
    elif (keyboard.is_pressed('W')):
        cmd = 1
    elif (keyboard.is_pressed('S')):
        cmd = -1
    #actualizar valor
    curtime = millis()
    if curtime > (oldtime + step):
        oldtime = curtime
        x = lectura_sensores()
        x["cmd"] = cmd
        x_scaled = pd.DataFrame(scaler.transform(x), index=x.index, columns=x.columns)
        output = dict(zip(out, mlp.predict(x_scaled[inp]).tolist()[0]))
        vehicle.control(
            throttle=output["throttle_input"],
            brake=output["brake_input"],
            clutch=output["clutch_input"],
            gear=round(output["gear"]*5)
        )
        display(x)
        clear_output(wait = True)

,throttle_input,brake_input,clutch_input,steering_input,gear,rpm,airspeed,accel,cmd
0,0.055812,0.00208,0.83782,0.0,1.0,3091.270996,4.087916,0.215858,0
